In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle
import glob

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

Read files

In [3]:
# Get the paths where all the files are stored
current_path = os.getcwd()
train_path = current_path + '/train.csv'
test_path = current_path + '/test.csv'

In [4]:
# read csv files
df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print('shape of train data:',df.shape)
print('shape of test data:',test_df.shape)

shape of train data: (42000, 785)
shape of test data: (28000, 784)


In [5]:
test_df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pixel84,pixel85,pixel86,pixel87,pixel88,pixel89,pixel90,pixel91,pixel92,pixel93,pixel94,pixel95,pixel96,pixel97,pixel98,pixel99,pixel100,pixel101,pixel102,pixel103,pixel104,pixel105,pixel106,pixel107,pixel108,pixel109,pixel110,pixel111,pixel112,pixel113,pixel114,pixel115,pixel116,pixel117,pixel118,pixel119,pixel120,pixel121,pixel122,pixel123,pixel124,pixel125,pixel126,pixel127,pixel128,pixel129,pixel130,pixel131,pixel132,pixel133,pixel134,pixel135,pixel136,pixel137,pixel138,pixel139,pixel140,pixel141,pixel142,pixel143,pixel144,pixel145,pixel146,pixel147,pixel148,pixel149,pixel150,pixel151,pixel152,pixel153,pixel154,pixel155,pixel156,pixel157,pixel158,pixel159,pixel160,pixel161,pixel162,pixel163,pixel164,pixel165,pixel166,pixel167,pixel168,pixel169,pixel170,pixel171,pixel172,pixel173,pixel174,pixel175,pixel176,pixel177,pixel178,pixel179,pixel180,pixel181,pixel182,pixel183,pixel184,pixel185,pixel186,pixel187,pixel188,pixel189,pixel190,pixel191,pixel192,pixel193,pixel194,pixel195,pixel196,pixel197,pixel198,pixel199,pixel200,pixel201,pixel202,pixel203,pixel204,pixel205,pixel206,pixel207,pixel208,pixel209,pixel210,pixel211,pixel212,pixel213,pixel214,pixel215,pixel216,pixel217,pixel218,pixel219,pixel220,pixel221,pixel222,pixel223,pixel224,pixel225,pixel226,pixel227,pixel228,pixel229,pixel230,pixel231,pixel232,pixel233,pixel234,pixel235,pixel236,pixel237,pixel238,pixel239,pixel240,pixel241,pixel242,pixel243,pixel244,pixel245,pixel246,pixel247,pixel248,pixel249,pixel250,pixel251,pixel252,pixel253,pixel254,pixel255,pixel256,pixel257,pixel258,pixel259,pixel260,pixel261,pixel262,pixel263,pixel264,pixel265,pixel266,pixel267,pixel268,pixel269,pixel270,pixel271,pixel272,pixel273,pixel274,pixel275,pixel276,pixel277,pixel278,pixel279,pixel280,pixel281,pixel282,pixel283,pixel284,pixel285,pixel286,pixel287,pixel288,pixel289,pixel290,pixel291,pixel292,pixel293,pixel294,pixel295,pixel296,pixel297,pixel298,pixel299,pixel300,pixel301,pixel302,pixel303,pixel304,pixel305,pixel306,pixel307,pixel308,pixel309,pixel310,pixel311,pixel312,pixel313,pixel314,pixel315,pixel316,pixel317,pixel318,pixel319,pixel320,pixel321,pixel322,pixel323,pixel324,pixel325,pixel326,pixel327,pixel328,pixel329,pixel330,pixel331,pixel332,pixel333,pixel334,pixel335,pixel336,pixel337,pixel338,pixel339,pixel340,pixel341,pixel342,pixel343,pixel344,pixel345,pixel346,pixel347,pixel348,pixel349,pixel350,pixel351,pixel352,pixel353,pixel354,pixel355,pixel356,pixel357,pixel358,pixel359,pixel360,pixel361,pixel362,pixel363,pixel364,pixel365,pixel366,pixel367,pixel368,pixel369,pixel370,pixel371,pixel372,pixel373,pixel374,pixel375,pixel376,pixel377,pixel378,pixel379,pixel380,pixel381,pixel382,pixel383,pixel384,pixel385,pixel386,pixel387,pixel388,pixel389,pixel390,pixel391,pixel392,pixel393,pixel394,pixel395,pixel396,pixel397,pixel398,pixel399,pixel400,pixel401,pixel402,pixel403,pixel404,pixel405,pixel406,pixel407,pixel408,pixel409,pixel410,pixel411,pixel412,pixel413,pixel414,pixel415,pixel416,pixel417,pixel418,pixel419,pixel420,pixel421,pixel422,pixel423,pixel424,pixel425,pixel426,pixel427,pixel428,pixel429,pixel430,pixel431,pixel432,pixel433,pixel434,pixel435,pixel436,pixel437,pixel438,pixel439,pixel440,pixel441,pixel442,pixel443,pixel444,pixel445,pixel446,pixel447,pixel448,pixel449,pixel450,pixel451,pixel452,pixel453,pixel454,pixel455,pixel

In [6]:
def make_img(list1):
    """
    Given a list which is the row from the dataframe 
    form the 28x28 image
    
    Input args:
    list1(List): row list of the dataframe
    out(np.array): Image of the size 28x28
    """
    l = len(list1)
    i = 0
    out = []
    while i < l:
        temp = list1[i:i+28]
        out.append(temp)
        i+=28
    return np.array(out)

In [7]:
def extract_image(df, is_train_data=True):
    """
    Given the dataframe extract the image from the rows
    
    Input Args:
    df(Pandas.DataFrame): Pandas dataframe where the first column is label
                            and rest are the pixel values
    is_train_data(bool): If True then the data generated is training data else its 
                        test image.
    
    Output Args:
    x(np.array): Image array with pixel values standardized from 0-1
    y(np.array): Labels array
    """
    x, y = [], []
    r,c = df.shape
    for i in range(r):
        if is_train_data:
            label = df.iloc[i,0]
            y.append(label)
            in_array = df.iloc[i,1:].to_list()
            x.append(make_img(in_array))
        else:
            in_array = df.iloc[i,:].to_list()
            x.append(make_img(in_array))
    # Convert list to arrays
    x, y = np.array(x), np.array(y)
    # Standardize the values of x from 0 to 1
    x = (np.max(x)-x)/(np.max(x)-np.min(x))
    return x, y

In [8]:
# Find sorted unique labels
uniq_labels = np.sort(df.label.unique())

In [9]:
# Extract individual df based on unique labels
df_list = []
for label in uniq_labels:
    dftemp = df[df.label == label]
    print(label,dftemp.shape)
    df_list.append(dftemp)

0 (4132, 785)
1 (4684, 785)
2 (4177, 785)
3 (4351, 785)
4 (4072, 785)
5 (3795, 785)
6 (4137, 785)
7 (4401, 785)
8 (4063, 785)
9 (4188, 785)


In [10]:
def create_dataset_for_training(df_list, num_dfs = 1, val_samples=400):
    """
    Create train dataframe and validation dataframe with randomization
    
    Input Args:
    df_list(List): List consisting of dataframes of unique labels
    num_dfs(int): Number of training-validation df pairs
    val_samples(int): Number of validation samples chosen from each subset of df
    
    Output Args:
    train_df_list(List): List of Dataframe used to create training samples
    val_df_list(List): List of Dataframe used to create validation samples
    """
    train_df_list = []
    val_df_list = []
    l = len(df_list)
    for _ in range(num_dfs):
        # Get the first dataframe as the start
        df = df_list[0]
        # shuffle the dataframe for randomization
        df = shuffle(df)
        # Create validation and training dataframe for 0th label
        val_df = df.iloc[:400]
        train_df = df.iloc[400:]
        for i in range(1,l):
            # Get the rest of the dataframes and randomize them
            df = df_list[i]
            df = shuffle(df)
            # Append the dataframes
            val_df = val_df.append(df.iloc[:400])
            train_df = train_df.append(df.iloc[400:])
        # Shuffle the result again
        train_df = shuffle(train_df)
        val_df = shuffle(val_df)
        # Append the result in the lists
        train_df_list.append(train_df)
        val_df_list.append(val_df)
    return train_df_list, val_df_list

In [11]:
train_df_list, val_df_list = create_dataset_for_training(df_list, 5, 400)

In [12]:
# Check if save path exists, if not then create one
save_path = current_path + '/Data'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

In [13]:
# Extract images from whole dataset
xtrain, ytrain = extract_image(df)
print('shape of training array is:', xtrain.shape)
print('shape of training label array is:', ytrain.shape)

shape of training array is: (42000, 28, 28)
shape of training label array is: (42000,)


In [14]:
# save the numpy arrays
print('Saving arrays...')
np.save(save_path+'/xtrain', xtrain)
np.save(save_path+'/ytrain', ytrain)

Saving arrays...


In [15]:
# Extract images from test dataframe
xtest, ytest = extract_image(test_df, False)
print('shape of training array is:', xtest.shape)
# save the numpy arrays
print('Saving arrays...')
np.save(save_path+'/xtest', xtest)

shape of training array is: (28000, 28, 28)
Saving arrays...


In [ ]:
# Generate the numpy images from the dataframes
for i in range(len(train_df_list)):
    # Get the individual dataframes
    train_df = train_df_list[i]
    val_df = val_df_list[i]
    # Extract images
    xtrain, ytrain = extract_image(train_df)
    xval, yval = extract_image(val_df)
    print('For sample no',i)
    print('shape of training array is:', xtrain.shape)
    print('shape of validation array is:', xval.shape)
    print('shape of training label array is:', ytrain.shape)
    print('shape of validation label array is:', yval.shape)
    # save the numpy arrays
    print('Saving arrays...')
    np.save(save_path+'/xtrain'+str(i), xtrain)
    np.save(save_path+'/ytrain'+str(i), ytrain)
    np.save(save_path+'/xval'+str(i), xval)
    np.save(save_path+'/yval'+str(i), yval)

For sample no 0
shape of training array is: (38000, 28, 28)
shape of validation array is: (4000, 28, 28)
shape of training label array is: (38000,)
shape of validation label array is: (4000,)
Saving arrays...
For sample no 1
shape of training array is: (38000, 28, 28)
shape of validation array is: (4000, 28, 28)
shape of training label array is: (38000,)
shape of validation label array is: (4000,)
Saving arrays...
For sample no 2
shape of training array is: (38000, 28, 28)
shape of validation array is: (4000, 28, 28)
shape of training label array is: (38000,)
shape of validation label array is: (4000,)
Saving arrays...
For sample no 3
shape of training array is: (38000, 28, 28)
shape of validation array is: (4000, 28, 28)
shape of training label array is: (38000,)
shape of validation label array is: (4000,)
Saving arrays...
For sample no 4
shape of training array is: (38000, 28, 28)
shape of validation array is: (4000, 28, 28)
shape of training label array is: (38000,)
shape of validat